In [ ]:
# Import libraries
import os
import re
from pathlib import Path

import numpy as np
import pandas as pd

# ruff: noqa

In [ ]:
# input variables

input_path = Path(
    "/Users/sailaja.yenepalli/Documents/scripts/a_Export Enhancements/TestLookUp/input"
)
output_path = Path(
    "/Users/sailaja.yenepalli/Documents/scripts/a_Export Enhancements/TestLookUp/output"
)

# input_path = Path('/Users/minji.kang/Documents/NGDT/Data_export_management/Report_CSV_Preprocessing_Generic_Script/MiResourceExtended/input/')
# output_path = Path('/Users/minji.kang/Documents/NGDT/Data_export_management/Report_CSV_Preprocessing_Generic_Script/MiResourceExtended/output/')

In [ ]:
def load_and_merge_response_files(input_dir):
    """Reads and combines all CSV files starting with 'report' from the specified directory."""
    try:
        # Find all files starting with 'report' in the directory
        report_files = input_dir.glob("report*.csv")

        # Read and combine CSV files on the fly
        combined_df = pd.concat(
            (pd.read_csv(file, encoding="ISO-8859-1") for file in report_files),
            ignore_index=True,
        )

        # Rename the first column
        combined_df.rename(
            columns={
                combined_df.columns[0]: "activity_submission_id",
                "activity_start_time": "activity_start_time_utc",
                "activity_end_time": "activity_end_time_utc",
                "activity_scheduled_time": "activity_scheduled_time_utc",
            },
            inplace=True,
        )

        return combined_df

    except FileNotFoundError:
        print(f"Error: Directory {input_dir} not found.")
    except pd.errors.EmptyDataError:
        print("Error: One or more files are empty.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    # Return an empty DataFrame if an error occurs
    return pd.DataFrame()


# Process the files using input path and save to output path
if input_path.exists():
    response_data = load_and_merge_response_files(input_path)
    if not response_data.empty:
        output_file = output_path / "report_all.csv"
        output_path.mkdir(parents=True, exist_ok=True)  # Ensure output directory exists
        response_data.to_csv(output_file, index=False)
        print(f"Combined report saved to: {output_file}")
    else:
        print("No data to combine or no matching files found.")
else:
    print(f"Input directory does not exist: {input_path}")

In [ ]:
def extract_applet_data_dict(data):
    """Extracts applet details and saves to a seperate file as applet dictionary"""
    return data[
        [
            "version",
            "activity_flow_id",
            "activity_flow_name",
            "activity_id",
            "activity_name",
            "item_id",
            "item",
            "prompt",
            "options",
        ]
    ].drop_duplicates()


# Process the response data and save applet data dictionary to CSV
applet_data_dict = extract_applet_data_dict(response_data)
applet_data_dict.to_csv(output_path / "applet_data_dict.csv", index=False)
print(f"Applet data dictionary saved to: {output_path / 'applet_data_dict.csv'}")

In [ ]:
def subscale_transform_long_format(data):
    """Transforms subscale columns into rows"""
    # Remove 'legacy_user_id' if it exists
    if "legacy_user_id" in data.columns:
        data = data.drop(columns=["legacy_user_id"])

    id_vars = data[
        [
            "activity_submission_id",
            "activity_flow_submission_id",
            "activity_scheduled_time_utc",
            "activity_start_time_utc",
            "activity_end_time_utc",
            "flag",
            "secret_user_id",
            "userId",
            "source_user_subject_id",
            "source_user_secret_id",
            "source_user_nickname",
            "source_user_relation",
            "source_user_tag",
            "target_user_subject_id",
            "target_user_secret_id",
            "target_user_nickname",
            "target_user_tag",
            "input_user_subject_id",
            "input_user_secret_id",
            "input_user_nickname",
            "activity_id",
            "activity_name",
            "activity_flow_id",
            "activity_flow_name",
            "version",
            "reviewing_id",
            "event_id",
            "timezone_offset",
        ]
    ].columns.to_list()

    value_vars = data.columns[data.columns.get_loc("timezone_offset") + 1 :].tolist()

    if not value_vars:  # Check if the list is empty
        print(" No Subscale Scores Present")
    else:
        # Reshape the DataFrame using melt for columns after 'timezone_offset'
        reshaped_data = data.melt(
            id_vars=id_vars,  # Columns to keep as identifiers
            value_vars=value_vars,  # Columns to reshape
            var_name="item",  # New column to hold column names
            value_name="response",  # New column to hold corresponding values
        ).dropna(subset=["response"])

        subscale_names = [
            x.replace("Optional text for ", "")
            for x in reshaped_data["item"].unique()
            if re.match(r"^Optional text for ", x)
            and x != "Optional text for Final SubScale Score"
        ]

        # Classify score types
        reshaped_data["score_type"] = reshaped_data["item"].apply(
            lambda x: "finalscore"
            if x == "Final SubScale Score"
            else "finalscore_text"
            if x == "Optional text for Final SubScale Score"
            #'lookup' if x in subscale_names else
            else "lookup_text"
            if re.match(r"^Optional text for ", x)
            else "subscale"
        )

        # Transform item names based on score types
        def transform_item(row):
            if row["score_type"] == "finalscore":
                return "activity_score"
            if row["score_type"] == "finalscore_text":
                return "activity_score_lookup_text"
            # elif row['score_type'] == 'lookup':
            #    return 'subscale_lookup_' + row['item']
            if row["score_type"] == "lookup_text":
                return "subscale_lookup_text_" + row["item"].replace(
                    "Optional text for ", ""
                )
            return "subscale_name_" + row["item"]

        reshaped_data["item"] = reshaped_data.apply(transform_item, axis=1)

        # Add additional computed columns
        reshaped_data = reshaped_data.drop(columns=["score_type"]).assign(
            item_id="", prompt="", options="", rawScore=""
        )

        # Prepare a subset of the original DataFrame for alignment
        subset_data = data[reshaped_data.columns.tolist()]

        # Combine the subset and reshaped DataFrame
        return pd.concat([subset_data, reshaped_data], axis=0, ignore_index=True)


subscale_tranformed_data_init = subscale_transform_long_format(response_data)
if subscale_tranformed_data_init is not None:
    subscale_tranformed_data_init.to_csv(
        output_path / "subscale_long_data.csv", index=False
    )
    subscale_tranformed_data = subscale_tranformed_data_init.copy()
else:
    subscale_tranformed_data = response_data.copy()

In [ ]:
def format_epochtime(data, column_name):
    """Convert epoch time in milliseconds to datetime."""
    return pd.to_datetime(
        pd.to_numeric(data[column_name], errors="coerce") / 1000, unit="s"
    )


# Apply the function to multiple columns using a loop
for col in [
    "activity_start_time_utc",
    "activity_end_time_utc",
    "activity_scheduled_time_utc",
]:
    subscale_tranformed_data[col] = format_epochtime(subscale_tranformed_data, col)

In [ ]:
# Process responses to clean and format time entries


def format_response(data):
    formatted_responses = []

    for i, row in data.iterrows():
        response = row.get("response", None)

        # Ensure response is a string or NaN
        if not isinstance(response, str):
            response = str(response) if not pd.isna(response) else np.nan

        # Clean responses
        if isinstance(response, str):
            if "geo:" in response:
                geo_match = re.search(
                    r"geo:\s*lat\s*\((.*?)\)\s*/\s*long\s*\((.*?)\)", response
                )
                if geo_match:
                    lat, long = geo_match.groups()
                    formatted_responses.append(f"{lat.strip()}/{long.strip()}")
                    continue

            if "value:" in response:
                formatted_responses.append(re.sub(r"value:\s*", "", response))
                continue

            if "date:" in response:
                formatted_responses.append(re.sub(r"date:\s*", "", response))
                continue

            if pd.isna(response):  # Handle NaN explicitly
                formatted_responses.append(np.nan)
                continue

            if "time:" in response:
                time_match = re.search(r"hr\s*(\d{1,2}),\s*min\s*(\d{1,2})", response)
                if time_match:
                    hour, minute = map(int, time_match.groups())
                    formatted_responses.append(f"{hour:02}:{minute:02}")
                    continue
                formatted_responses.append(np.nan)
                continue

            if "time_range:" in response:
                try:
                    clean_time = re.sub(r"[a-zA-Z\s+(\)_:]", "", response).replace(
                        ",", ":"
                    )
                    time_parts = clean_time.split("/")
                    formatted_parts = [
                        f"{part.split(':')[0].zfill(2)}:{part.split(':')[1].zfill(2)}"
                        for part in time_parts
                    ]
                    formatted_responses.append("/".join(formatted_parts))
                except (IndexError, ValueError):
                    formatted_responses.append(np.nan)
                continue

        # Fallback case
        formatted_responses.append(response)

    return pd.Series(formatted_responses)


subscale_tranformed_data["formatted_response"] = format_response(
    subscale_tranformed_data
)

In [ ]:
# test = timestamp_conv_data.copy()
# test = test[test['activity_submission_id']=='5bc2fa7a-49db-49b8-b672-685dfbb2f899'].reset_index()

In [ ]:
# Maps responses to scores based on the options column in the DataFrame.


def response_value_score_mapping(data):
    response_scores = []
    response_values = []

    for options, response in zip(data["options"], data["response"], strict=False):
        # Ensure 'options' and 'response' are valid strings
        if not isinstance(options, str) or not isinstance(response, str):
            response_scores.append(np.nan)
            response_values.append(np.nan)
            continue

        # Check if options contain scores
        if "score: " in options:
            # Parse options and responses
            split_options = [
                opt.strip() for opt in options.strip().split("),") if "(score" in opt
            ]
            split_response = [
                resp.strip() for resp in response.strip().split(": ")[1].split(",")
            ]

            # Build the score mapping dictionary
            scores = {
                opt.split(": ")[1].split(" ")[0]:  # Extract position part
                opt.split("score: ")[1].strip(" )")  # Extract score part
                for opt in split_options
                if "score: " in opt
            }

            # Map responses to scores
            response_score_mapping = [
                scores.get(resp, "N/A") for resp in split_response
            ]
            response_scores.append(", ".join(response_score_mapping))
            response_values.append(np.nan)

        elif ": " in options:
            if "Min: 0, Max:" in options:
                max_value = re.sub(r"Min: 0, Max: ", "", options)
                max_value = int(max_value)

                if max_value > 1:
                    slider_response = re.sub("value: ", "", response)
                    response_values.append(", ".join(slider_response))
                    response_scores.append(np.nan)

            else:
                value_options = ", " + options + ","
                split_options_text = [
                    opt.strip() for opt in re.findall(r",\s(.*?):", value_options)
                ]
                split_options_value = [
                    opt.strip() for opt in re.findall(r":\s(\d+),", value_options)
                ]
                split_response_values = [
                    resp.strip() for resp in response.strip().split(": ")[1].split(",")
                ]

                # Build actual response mapping
                values = {
                    value: text  # Map position (value) to response text
                    for text, value in zip(
                        split_options_text, split_options_value, strict=False
                    )
                }

                # Map response positions to actual values
                response_value_mapping = [
                    values.get(resp, re.sub("value: ", "", response))
                    for resp in split_response_values
                ]
                response_values.append(", ".join(response_value_mapping))
                response_scores.append(np.nan)

        else:
            response_scores.append(np.nan)
            response_values.append(np.nan)

    return pd.Series(response_values), pd.Series(response_scores)


(
    subscale_tranformed_data["response_values"],
    subscale_tranformed_data["response_scores"],
) = response_value_score_mapping(subscale_tranformed_data)

In [ ]:
test_final = subscale_tranformed_data.copy()
test_final["merged_responses"] = (
    test_final["response_scores"]
    .combine_first(test_final["response_values"])
    .combine_first(test_final["formatted_response"])
)
check_df = test_final[
    [
        "userId",
        "activity_id",
        "activity_name",
        "item_id",
        "item",
        "response",
        "options",
        "response_scores",
        "response_values",
        "formatted_response",
        "merged_responses",
    ]
]
check_df.to_csv(
    os.path.join(output_path, "CHECK_response_value_score_mapping.csv"), index=False
)

In [ ]:
# Define column list and response column name
mycolumn_list = [
    "userId",
    "secret_user_id",
    "source_user_secret_id",
    "target_user_secret_id",
    "input_user_secret_id",
    "activity_start_time_utc",
    "activity_end_time_utc",
    "activity_scheduled_time_utc",
    "activity_flow_id",
    "activity_flow_name",
    "activity_id",
    "activity_name",
    "event_id",
    "version",
]


def widen_data(data, column_list):
    """Transforms data into a wide format based on the specified column list."""
    # merge formatted response, values and scores created a single response field
    data = data.copy()
    data["merged_responses"] = (
        data["response_scores"]
        .combine_first(data["response_values"])
        .combine_first(data["formatted_response"])
    )

    # Convert datetime columns to string and handle NaT
    datetime_cols = data.select_dtypes(include=["datetime"]).columns
    data[datetime_cols] = data[datetime_cols].astype(str).replace("NaT", "")

    # Fill missing values in specified columns
    data[column_list] = data[column_list].fillna("")

    # Group by the column list and combine IDs
    answers = (
        data.groupby(column_list)["activity_submission_id"]
        .apply(lambda x: "|".join(x.astype(str)))
        .reset_index()
    )

    # Create combined column names
    data["combined_cols"] = (
        "activityName["
        + data["activity_name"]
        + "]_itemName["
        + data["item"].astype(str)
        + "]_itemId["
        + data["item_id"].astype(str)
        + "]"
    )
    data["combined_cols"] = np.where(
        data["combined_cols"].str.contains("_itemId[]", regex=False),
        data["combined_cols"].str.replace("_itemId[]", "", regex=False),
        data["combined_cols"],
    )

    # Select relevant columns for pivoting
    subset_columns = column_list + ["combined_cols", "merged_responses"]
    dat_subset = data[subset_columns]

    # Pivot the data into wide format
    dat_wide = pd.pivot_table(
        dat_subset,
        index=column_list,
        columns="combined_cols",
        values="merged_responses",
        aggfunc="last",
    ).reset_index()

    # Merge with the combined IDs
    dat_wide = pd.merge(dat_wide, answers, on=column_list, how="outer")

    return dat_wide


# Apply the function to process data into wide format
data_wide = widen_data(subscale_tranformed_data, mycolumn_list)

# Save the output to CSV
data_wide.to_csv(os.path.join(output_path, "data_wide_all.csv"), index=False)